In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
%pip install --upgrade pip

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
%pip install nltk

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
jigsaw_df = spark.read.option("header", True).option("multiline", True).option("escape", "\\").csv("/FileStore/tables/train.csv")


In [0]:
display(jigsaw_df)

id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0000997932d777bf,"Explanation Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,0,0,0,0,0
000103f0d9cfb60f,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,0,0,0,0,0
000113f07ec002fd,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,0,0,0,0,0
0001b41b1c6bb37e,"""",null,null,null,null,null,null
More,null,null,null,null,null,null,null
I can't make any real suggestions on improvement - I wondered if the section statistics should be later on,"or a subsection of """"""""types of accidents"""""""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on",if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.,null,null,null,null,null
"There appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """"""",0,0,0,0,0,0,null
0001d958c54c6e35,"You, sir, are my hero. Any chance you remember what page that's on?",0,0,0,0,0,0
00025465d4725e87,"""",null,null,null,null,null,null
Congratulations from me as well,"use the tools well. · talk """"""",0,0,0,0,0,0


In [0]:
from pyspark.ml.feature import RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, StringType


tokenizer = RegexTokenizer(inputCol="comment_text", outputCol="comment_tokens", pattern=r"\W", toLowercase=True)


jigsaw_df = tokenizer.transform(jigsaw_df)


def to_string_list(tokens):
    return [str(token) for token in tokens]


to_string_list_udf = udf(to_string_list, ArrayType(StringType()))


jigsaw_df = jigsaw_df.withColumn("comment_tokens", to_string_list_udf(col("comment_tokens")))


In [0]:
display(spark_df)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-501399624115996> in <cell line: 1>()
----> 1 display(spark_df)

NameError: name 'spark_df' is not defined

In [0]:

tokenizer1 = RegexTokenizer(inputCol="comment_text", outputCol="comment_tokens1", pattern=r"\W+", toLowercase=True)
tokenizer2 = RegexTokenizer(inputCol="comment_text", outputCol="comment_tokens2", pattern=r"\s+", toLowercase=True)


jigsaw_df = tokenizer1.transform(jigsaw_df)
jigsaw_df = tokenizer2.transform(jigsaw_df)

def to_string_list(tokens):
    return [str(token) for token in tokens]


to_string_list_udf = udf(to_string_list, ArrayType(StringType()))


jigsaw_df = jigsaw_df.withColumn("comment_tokens1", to_string_list_udf(col("comment_tokens1")))
jigsaw_df = jigsaw_df.withColumn("comment_tokens2", to_string_list_udf(col("comment_tokens2")))


In [0]:
from pyspark.ml.feature import StopWordsRemover

# definir o StopWordsRemover
stopwords = StopWordsRemover.loadDefaultStopWords("english")
remover = StopWordsRemover(inputCol="comment_tokens", outputCol="comment_tokens_sw", stopWords=stopwords)

# aplicar o StopWordsRemover ao dataframe
jigsaw_df = remover.transform(jigsaw_df)


In [0]:

display(jigsaw_df)

In [0]:
from nltk.stem.porter import PorterStemmer
from pyspark.ml import Transformer
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

class Stemmer(Transformer):
    def __init__(self, inputCol, outputCol=None):
        super(Stemmer, self).__init__()
        self.input_column = inputCol
        self.output_column = outputCol or inputCol

    def _transform(self, df):
        stemmer_udf = udf(lambda x: [PorterStemmer().stem(y) for y in x], ArrayType(StringType()))
        return df.withColumn(self.output_column, stemmer_udf(self.input_column))



class Stemmer(Transformer):
    def __init__(self, inputCol, outputCol=None):
        super(Stemmer, self).__init__()
        self.input_column = inputCol
        self.output_column = outputCol or inputCol

    def _transform(self, df):
        stemmer_udf = udf(lambda x: [PorterStemmer().stem(y) for y in x], ArrayType(StringType()))
        return df.withColumn(self.output_column, stemmer_udf(self.input_column))


stemmer = Stemmer(inputCol="comment_tokens_sw", outputCol="comment_tokens_sw_stem")
jigsaw_df = stemmer.transform(jigsaw_df)



In [0]:

display(jigsaw_df)

In [0]:


tokenizer = RegexTokenizer(inputCol="comment_text", outputCol="comment_tokens", pattern=r"\W", toLowercase=True)
tokenizer2 = RegexTokenizer(inputCol="comment_text", outputCol="comment_tokens_2", pattern=r"\d", toLowercase=True)
remover = StopWordsRemover(inputCol="comment_tokens_sw", outputCol="comment_tokens_sw_removed")
remover2 = StopWordsRemover(inputCol="comment_tokens_2_sw", outputCol="comment_tokens_2_sw_removed")
stemmer = Stemmer(inputCol="comment_tokens_sw_removed", outputCol="comment_tokens_sw_stem")
stemmer2 = Stemmer(inputCol="comment_tokens_2_sw_removed", outputCol="comment_tokens_2_sw_stem")


pipeline = Pipeline(stages=[tokenizer, remover, stemmer, tokenizer2, remover2, stemmer2])


jigsaw_df_transformed = pipeline.fit(jigsaw_df).transform(jigsaw_df)


concat_udf = udf(lambda x, y: x + y, ArrayType(StringType()))
jigsaw_df_transformed = jigsaw_df_transformed.withColumn("comment_transformed", concat_udf("comment_tokens_sw_stem", "comment_tokens_2_sw_stem"))


In [0]:
display(jigsaw_df)